In [5]:
import numpy as np  
import pandas as pd 
import os, csv, sys
import glob
import subprocess
import navpy
from tqdm.notebook import tqdm
from dataclasses import dataclass
from scipy.interpolate import InterpolatedUnivariateSpline
from datetime import datetime, timezone, timedelta
import pandas as pd
import pymap3d as pm
import pymap3d.vincenty as pmv
import glob as gl
import numpy as np
import scipy.optimize
from tqdm.auto import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.spatial import distance
from time import time 
from pathlib import Path
# Constants
CLIGHT = 299_792_458   # speed of light (m/s)
RE_WGS84 = 6_378_137   # earth semimajor axis (WGS84) (m)
OMGE = 7.2921151467E-5  # earth angular velocity (IS-GPS) (rad/s)

folder_path = os.path.abspath('gnss_analysis')
sys.path.append(folder_path)
from gnssutils import EphemerisManager
from libWLS import WLS
from kalman_filter import kalman_smoothing

c:\Users\luan1\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'requests'

In [2]:
path = 'gnss_2024_06_01_12_00_xm.csv'
gnss = pd.read_csv(path, index_col=False)
filtered_gnss = gnss[gnss['ReceivedSvTimeNanos'] > 1e10]
measurement = filtered_gnss

In [3]:
# Lọc các vệ tinh thuộc GPS (ConstellationType == 1)
# Loc cac ve tinh co Multipath
measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'

measurement = measurement.loc[measurement['MultipathIndicator'] == 0]

measurement = measurement.loc[measurement['ReceivedSvTimeUncertaintyNanos'] < 100]

# Chỉ lấy các vệ tinh bắt GPS
measurement = measurement.loc[measurement['Constellation'] == 'G']
# Đảm bảo cột 'Svid' là chuỗi ký tự để có thể sử dụng hàm str.len()
measurement['Svid'] = measurement['Svid'].astype(str)
# Thêm số 0 vào trước 'Svid' nếu chiều dài của nó là 1
measurement.loc[measurement['Svid'].str.len() == 1, 'Svid'] = '0' + measurement['Svid']
# Tạo cột mới 'SvName' bằng cách nối 'Constellation' và 'Svid'
measurement['SvName'] = measurement['Constellation'] + measurement['Svid']

measurement = measurement.loc[measurement['CarrierFrequencyHz'] > 1275420030]

measurement

C:\Users\luan1\AppData\Local\Temp\ipykernel_12464\654579464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'


,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,SatelliteInterSignalBiasUncertaintyNanos,CodeType,ChipsetElapsedRealtimeNanos,SignalType,ArrivalTimeNanosSinceGpsEpoch,ReceivedSvTimeNanosSinceGpsEpoch,RawPseudorangeMeters,RawPseudorangeUncertaintyMeters,Constellation,SvName
143,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,0.0,C,557820104692300,GPS,1.401253e+18,1.401253e+18,2.325737e+07,6.295642,G,G05
144,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,0.0,C,557820104692300,GPS,1.401253e+18,1.401253e+18,2.183594e+07,3.597509,G,G06
145,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,0.0,C,557820104692300,GPS,1.401253e+18,1.401253e+18,2.352453e+07,9.293566,G,G09
146,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,0.0,C,557820104692300,GPS,1.401253e+18,1.401253e+18,2.178606e+07,2.098547,G,G11
147,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,0.0,C,557820104692300,GPS,1.401253e+18,1.401253e+18,2.294087e+07,3.597509,G,G12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6279,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,0.0,C,558028907831626,GPS,1.401253e+18,1.401253e+18,2.284632e+07,9.293566,G,G12
6280,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,0.0,C,558028907831626,GPS,1.401253e+18,1.401253e+18,2.414503e+07,11.991698,G,G14
6281,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,0.0,C,558028907831626,GPS,1.401253e+18,1.401253e+18,2.217800e+07,3.297717,G,G17
6282,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,0.0,C,558028907831626,GPS,1.401253e+18,1.401253e+18,2.077814e+07,9.293566,G,G19


In [4]:
#chuyển giá trị thời gian về dạng date time
measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
#Chia measurement thành các epoch, mỗi epoch gồm các vệ tinh có cùng thời gian thu
measurement['Epoch'] = 0
measurement.loc[measurement['UnixTime'] - measurement['UnixTime'].shift() > timedelta(milliseconds=200), 'Epoch'] = 1
measurement['Epoch'] = measurement['Epoch'].cumsum()
measurement

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,ChipsetElapsedRealtimeNanos,SignalType,ArrivalTimeNanosSinceGpsEpoch,ReceivedSvTimeNanosSinceGpsEpoch,RawPseudorangeMeters,RawPseudorangeUncertaintyMeters,Constellation,SvName,UnixTime,Epoch
143,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.325737e+07,6.295642,G,G05,2024-06-01 05:00:55+00:00,0
144,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.183594e+07,3.597509,G,G06,2024-06-01 05:00:55+00:00,0
145,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.352453e+07,9.293566,G,G09,2024-06-01 05:00:55+00:00,0
146,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.178606e+07,2.098547,G,G11,2024-06-01 05:00:55+00:00,0
147,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.294087e+07,3.597509,G,G12,2024-06-01 05:00:55+00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6279,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,558028907831626,GPS,1.401253e+18,1.401253e+18,2.284632e+07,9.293566,G,G12,2024-06-01 05:04:24+00:00,209
6280,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,558028907831626,GPS,1.401253e+18,1.401253e+18,2.414503e+07,11.991698,G,G14,2024-06-01 05:04:24+00:00,209
6281,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,558028907831626,GPS,1.401253e+18,1.401253e+18,2.217800e+07,3.297717,G,G17,2024-06-01 05:04:24+00:00,209
6282,1717218264000,156048122000000,18,0.0,-1401097434289769506,0.529009,52.973377,8.418931,14.977861,196.0,...,558028907831626,GPS,1.401253e+18,1.401253e+18,2.077814e+07,9.293566,G,G19,2024-06-01 05:04:24+00:00,209


In [5]:
# Khởi tạo biến để lưu trữ kết quả
filtered_measurement = pd.DataFrame()

# Duyệt qua từng epoch
for epoch in measurement['Epoch'].unique():
    one_epoch = measurement.loc[measurement['Epoch'] == epoch].drop_duplicates(subset='SvName')
    # Thêm kết quả vào DataFrame kết quả
    filtered_measurement = pd.concat([filtered_measurement, one_epoch])

# Reset index của DataFrame mới
filtered_measurement.reset_index(drop=True, inplace=True)
filtered_measurement = filtered_measurement.groupby('Epoch').apply(lambda x: x.sort_values(by='Svid')).reset_index(drop=True)
measurement = filtered_measurement
measurement.head(20)

C:\Users\luan1\AppData\Local\Temp\ipykernel_12464\2289186973.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_measurement = filtered_measurement.groupby('Epoch').apply(lambda x: x.sort_values(by='Svid')).reset_index(drop=True)


,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,ChipsetElapsedRealtimeNanos,SignalType,ArrivalTimeNanosSinceGpsEpoch,ReceivedSvTimeNanosSinceGpsEpoch,RawPseudorangeMeters,RawPseudorangeUncertaintyMeters,Constellation,SvName,UnixTime,Epoch
0,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.325737e+07,6.295642,G,G05,2024-06-01 05:00:55+00:00,0
1,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.183594e+07,3.597509,G,G06,2024-06-01 05:00:55+00:00,0
2,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.352453e+07,9.293566,G,G09,2024-06-01 05:00:55+00:00,0
3,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.178606e+07,2.098547,G,G11,2024-06-01 05:00:55+00:00,0
4,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.294087e+07,3.597509,G,G12,2024-06-01 05:00:55+00:00,0
5,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.399951e+07,9.293566,G,G14,2024-06-01 05:00:55+00:00,0
6,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.213817e+07,10.492736,G,G17,2024-06-01 05:00:55+00:00,0
7,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.077139e+07,7.195019,G,G19,2024-06-01 05:00:55+00:00,0
8,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.104215e+07,6.295642,G,G20,2024-06-01 05:00:55+00:00,0
9,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,557820104692300,GPS,1.401253e+18,1.401253e+18,2.216618e+07,13.790453,G,G22,2024-06-01 05:00:55+00:00,0


In [6]:
#lấy dữ liệu ephemeris
ephemeris_data_directory = 'output'
manager = EphemerisManager(ephemeris_data_directory)

# while num_sat < 4:
#     one_epoch = measurement.loc[measurement['Epoch'] == epoch].drop_duplicates(subset='SvName')
    
#     if not one_epoch.empty:
#         timestamp = pd.to_datetime(one_epoch.iloc[0]['UnixTime'], unit='s')
#         one_epoch.set_index('SvName', inplace=True)
#         num_sat = len(one_epoch.index)
    
#     epoch += 1

measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
timestamps = pd.to_datetime(measurement['UnixTime'].unique(),unit='s')
ephemeris_data = pd.DataFrame()

for timestamp in timestamps:
    # Get satellite names for the current timestamp
    sats = measurement.loc[measurement['UnixTime'] == timestamp, 'SvName'].tolist()
    # Retrieve ephemeris data
    ephemeris = manager.get_ephemeris(timestamp, sats)
    # Append the retrieved ephemeris data to the list
    ephemeris_data = pd.concat([ephemeris_data,ephemeris])

#ephemeris = manager.get_ephemeris(timestamp, sats)
#timestamp
ephemeris_data.reset_index(drop=True, inplace=True)
ephemeris_data.head(20)

,time,SVclockBias,SVclockDrift,SVclockDriftRate,IODE,C_rs,deltaN,M_0,C_uc,e,...,L2Pflag,SVacc,health,TGD,IODC,TransTime,FitIntvl,source,t_oc,Leap Seconds
0,2024-06-01 04:00:00+00:00,-0.000175,-1.364242e-12,0.0,66.0,30.06250,4.409827e-09,-2.248861,1.432374e-06,0.005883,...,0.0,2.0,0.0,-1.071021e-08,66.0,525618.0,4.0,output\nasa\brdc1530.24n,532800.0,18
1,2024-06-01 04:00:00+00:00,0.000243,-2.046363e-11,0.0,21.0,-10.50000,3.498003e-09,1.688643,-5.122274e-07,0.002969,...,0.0,2.0,0.0,3.725290e-09,21.0,525618.0,4.0,output\nasa\brdc1530.24n,532800.0,18
2,2024-06-01 04:00:00+00:00,0.000218,1.432454e-11,0.0,35.0,-52.28125,5.035210e-09,-2.153307,-2.609566e-06,0.002720,...,0.0,2.0,0.0,9.313226e-10,35.0,529098.0,4.0,output\nasa\brdc1530.24n,532800.0,18
3,2024-06-01 04:00:00+00:00,-0.000676,-8.640200e-12,0.0,35.0,-13.25000,4.030525e-09,2.975690,-5.327165e-07,0.001317,...,0.0,2.0,0.0,-8.847564e-09,291.0,525618.0,4.0,output\nasa\brdc1530.24n,532800.0,18
4,2024-06-01 04:00:00+00:00,-0.000510,-3.296918e-12,0.0,29.0,73.25000,4.664480e-09,0.082297,3.790483e-06,0.008772,...,0.0,2.0,0.0,-1.257285e-08,29.0,525620.0,4.0,output\nasa\brdc1530.24n,532800.0,18
5,2024-06-01 04:00:00+00:00,0.000415,9.208634e-12,0.0,176.0,73.06250,5.061997e-09,0.062022,3.620982e-06,0.004533,...,0.0,2.0,0.0,-7.916242e-09,432.0,529422.0,4.0,output\nasa\brdc1530.24n,532800.0,18
6,2024-06-01 03:59:44+00:00,0.000700,-4.547474e-12,0.0,6.0,-19.21875,4.392683e-09,-2.617446,-9.350479e-07,0.013399,...,0.0,2.0,0.0,-1.117587e-08,6.0,527268.0,4.0,output\nasa\brdc1530.24n,532784.0,18
7,2024-06-01 04:00:00+00:00,0.000490,5.343281e-12,0.0,90.0,-27.31250,4.448400e-09,-0.624906,-1.451001e-06,0.009877,...,0.0,2.0,0.0,-1.536682e-08,90.0,525618.0,4.0,output\nasa\brdc1530.24n,532800.0,18
8,2024-06-01 04:00:00+00:00,0.000375,-1.364242e-12,0.0,96.0,34.93750,4.721268e-09,2.106423,1.870096e-06,0.003586,...,0.0,2.0,0.0,-8.381903e-09,96.0,525618.0,4.0,output\nasa\brdc1530.24n,532800.0,18
9,2024-06-01 04:00:00+00:00,-0.000021,-5.115908e-12,0.0,95.0,76.34375,4.599834e-09,-2.094141,3.982335e-06,0.014274,...,0.0,2.0,0.0,-1.117587e-08,95.0,528588.0,4.0,output\nasa\brdc1530.24n,532800.0,18


In [7]:
LIGHTSPEED = 2.99792458e8

def calculate_satellite_position(ephemeris, transmit_time):
    mu = 3.986005e14
    OmegaDot_e = 7.2921151467e-5
    F = -4.442807633e-10
    sv_position = pd.DataFrame()
    sv_position['sv']= ephemeris.index
    sv_position.set_index('sv', inplace=True)
    sv_position['t_k'] = transmit_time - ephemeris['t_oe']
    A = ephemeris['sqrtA'].pow(2)
    n_0 = np.sqrt(mu / A.pow(3))
    n = n_0 + ephemeris['deltaN']
    M_k = ephemeris['M_0'] + n * sv_position['t_k']
    E_k = M_k
    err = pd.Series(data=[1]*len(sv_position.index))
    i = 0
    while err.abs().min() > 1e-8 and i < 10:
        new_vals = M_k + ephemeris['e']*np.sin(E_k)
        err = new_vals - E_k
        E_k = new_vals
        i += 1
        
    sinE_k = np.sin(E_k)
    cosE_k = np.cos(E_k)
    delT_r = F * ephemeris['e'].pow(ephemeris['sqrtA']) * sinE_k
    delT_oc = transmit_time - ephemeris['t_oc']
    sv_position['delT_sv'] = ephemeris['SVclockBias'] + ephemeris['SVclockDrift'] * delT_oc + ephemeris['SVclockDriftRate'] * delT_oc.pow(2)

    v_k = np.arctan2(np.sqrt(1-ephemeris['e'].pow(2))*sinE_k,(cosE_k - ephemeris['e']))

    Phi_k = v_k + ephemeris['omega']

    sin2Phi_k = np.sin(2*Phi_k)
    cos2Phi_k = np.cos(2*Phi_k)

    du_k = ephemeris['C_us']*sin2Phi_k + ephemeris['C_uc']*cos2Phi_k
    dr_k = ephemeris['C_rs']*sin2Phi_k + ephemeris['C_rc']*cos2Phi_k
    di_k = ephemeris['C_is']*sin2Phi_k + ephemeris['C_ic']*cos2Phi_k

    u_k = Phi_k + du_k

    r_k = A*(1 - ephemeris['e']*np.cos(E_k)) + dr_k

    i_k = ephemeris['i_0'] + di_k + ephemeris['IDOT']*sv_position['t_k']

    x_k_prime = r_k*np.cos(u_k)
    y_k_prime = r_k*np.sin(u_k)

    Omega_k = ephemeris['Omega_0'] + (ephemeris['OmegaDot'] - OmegaDot_e)*sv_position['t_k'] - OmegaDot_e*ephemeris['t_oe']

    sv_position['x_k'] = x_k_prime*np.cos(Omega_k) - y_k_prime*np.cos(i_k)*np.sin(Omega_k)
    sv_position['y_k'] = x_k_prime*np.sin(Omega_k) + y_k_prime*np.cos(i_k)*np.cos(Omega_k)
    sv_position['z_k'] = y_k_prime*np.sin(i_k)
    sv_position['SvClockBiasMeters'] = ephemeris['SVclockBias'] * LIGHTSPEED
    sv_position['SvClockDriftMetersPerSecond'] = ephemeris['SVclockDrift'] * LIGHTSPEED

    #calculate velocity
    Edot_k = n_0/(1-ephemeris['e']*np.cos(E_k))
    
    vdot_k = Edot_k * np.sqrt(1 - ephemeris['e']**2)/(1 - ephemeris['e']*np.cos(E_k))
    
    d_idotk = ephemeris['IDOT'] + 2 * vdot_k * (ephemeris['C_is'] * cos2Phi_k - ephemeris['C_ic']*sin2Phi_k)
    
    udot_k = vdot_k + 2 * vdot_k * (ephemeris['C_us']*cos2Phi_k - ephemeris['C_uc']*sin2Phi_k)
    
    rdot_k = ephemeris['e'] * A * Edot_k * np.sin(E_k) + 2 * vdot_k * (ephemeris['C_rs'] * cos2Phi_k - ephemeris['C_rc'] * sin2Phi_k)
    
    Omegadot_k = ephemeris['OmegaDot'] - OmegaDot_e
    
    x_plus_dot_k = rdot_k * np.cos(u_k) - r_k * udot_k*np.sin(u_k)
    y_plus_dot_k = rdot_k * np.sin(u_k) + r_k * udot_k*np.cos(u_k)
    
    sv_position['v_x'] = -x_k_prime * Omegadot_k * np.sin(Omega_k) + x_plus_dot_k * np.cos(Omega_k) - y_plus_dot_k * np.sin(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.cos(Omega_k) * np.cos(i_k) - d_idotk * np.sin(Omega_k)*np.sin(i_k))
    
    sv_position['v_y'] = x_k_prime * Omegadot_k * np.cos(Omega_k) + x_plus_dot_k * np.sin(Omega_k) + y_plus_dot_k * np.cos(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.sin(Omega_k) * np.cos(i_k) + d_idotk * np.cos(Omega_k)*np.sin(i_k))
    
    sv_position['v_z'] = y_plus_dot_k * np.sin(i_k) + y_k_prime * d_idotk * np.cos(i_k)

    return sv_position

# Run the function and check out the results:
measurement['ReceivedSvTimeNanos'] = pd.to_numeric(measurement['ReceivedSvTimeNanos'])
measurement['TimeOffsetNanos'] = pd.to_numeric(measurement['TimeOffsetNanos'])
sv_position = calculate_satellite_position(ephemeris_data, (measurement['ReceivedSvTimeNanos'] + measurement['TimeOffsetNanos'])*1e-9)
#xs = sv_position[['x_k', 'y_k', 'z_k']].to_numpy()
measurement['SvPositionXEcefMeters'] = sv_position['x_k']
measurement['SvPositionYEcefMeters'] = sv_position['y_k']
measurement['SvPositionZEcefMeters'] = sv_position['z_k']
measurement['SvVelocityXEcefMetersPerSecond'] = sv_position['v_x']
measurement['SvVelocityYEcefMetersPerSecond'] = sv_position['v_y']
measurement['SvVelocityZEcefMetersPerSecond'] = sv_position['v_z']
measurement['SvClockBiasMeters'] = sv_position['SvClockBiasMeters']
measurement['SvClockDriftMetersPerSecond'] = sv_position['SvClockDriftMetersPerSecond']
measurement.head(20)

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,UnixTime,Epoch,SvPositionXEcefMeters,SvPositionYEcefMeters,SvPositionZEcefMeters,SvVelocityXEcefMetersPerSecond,SvVelocityYEcefMetersPerSecond,SvVelocityZEcefMetersPerSecond,SvClockBiasMeters,SvClockDriftMetersPerSecond
0,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,6.136833e+06,2.389080e+07,-9.910740e+06,-255.830133,1227.456085,2860.727050,-52385.274451,-0.000409
1,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-7.683355e+06,1.244575e+07,2.222718e+07,-2297.156235,-1600.427343,112.955590,72877.553749,-0.006135
2,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-2.456996e+07,6.538651e+06,7.694324e+06,755.314025,-689.494610,2961.296046,65263.395541,0.004294
3,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,5.619935e+06,1.929543e+07,1.741586e+07,-1854.425464,-1200.305214,1925.372281,-202532.950758,-0.002590
4,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,1.118836e+07,1.459532e+07,1.889999e+07,-487.793893,2412.400104,-1546.246324,-152978.660031,-0.000988
5,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-1.320274e+07,1.645083e+07,-1.597681e+07,-31.697490,-2060.758941,-2112.076678,124300.692994,0.002761
6,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-2.217486e+07,1.363012e+07,6.117261e+06,-751.046171,62.519383,-3057.228768,210002.202358,-0.001363
7,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-1.602253e+07,1.574877e+07,1.367178e+07,-1754.387150,381.230152,-2501.715346,146925.109697,0.001602
8,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,1.931277e+06,2.655191e+07,1.118846e+06,-302.017598,-103.987044,3146.603579,112280.424609,-0.000409
9,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-1.007453e+07,2.291824e+07,-8.875482e+06,-46.641740,-1200.902714,-2882.426453,-6410.232785,-0.001534


In [8]:
measurement.to_csv('gnss_sat.csv')

In [9]:
epoch = 0
num_sat = 0

while num_sat < 4:
    one_epoch = measurement.loc[measurement['Epoch'] == epoch]
    if not one_epoch.empty:
        one_epoch.set_index('SvName', inplace=True)
        num_sat = len(one_epoch.index)
    epoch += 1
one_epoch

,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,UnixTime,Epoch,SvPositionXEcefMeters,SvPositionYEcefMeters,SvPositionZEcefMeters,SvVelocityXEcefMetersPerSecond,SvVelocityYEcefMetersPerSecond,SvVelocityZEcefMetersPerSecond,SvClockBiasMeters,SvClockDriftMetersPerSecond
SvName,,,,,,,,,,,,,,,,,,,,,
G05,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,6.136833e+06,2.389080e+07,-9.910740e+06,-255.830133,1227.456085,2860.727050,-52385.274451,-0.000409
G06,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-7.683355e+06,1.244575e+07,2.222718e+07,-2297.156235,-1600.427343,112.955590,72877.553749,-0.006135
G09,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-2.456996e+07,6.538651e+06,7.694324e+06,755.314025,-689.494610,2961.296046,65263.395541,0.004294
G11,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,5.619935e+06,1.929543e+07,1.741586e+07,-1854.425464,-1200.305214,1925.372281,-202532.950758,-0.002590
G12,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,1.118836e+07,1.459532e+07,1.889999e+07,-487.793893,2412.400104,-1546.246324,-152978.660031,-0.000988
G14,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-1.320274e+07,1.645083e+07,-1.597681e+07,-31.697490,-2060.758941,-2112.076678,124300.692994,0.002761
G17,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-2.217486e+07,1.363012e+07,6.117261e+06,-751.046171,62.519383,-3057.228768,210002.202358,-0.001363
G19,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,-1.602253e+07,1.574877e+07,1.367178e+07,-1754.387150,381.230152,-2501.715346,146925.109697,0.001602
G20,1717218055000,155839302000000,18,0.0,-1401097434289768911,0.755893,45.585781,-12.019147,9.190056,196.0,...,2024-06-01 05:00:55+00:00,0,1.931277e+06,2.655191e+07,1.118846e+06,-302.017598,-103.987044,3146.603579,112280.424609,-0.000409


In [10]:
wls = WLS()
ecef_list = []
for epoch in measurement['Epoch'].unique():
    one_epoch = measurement.loc[(measurement['Epoch'] == epoch)] 
    if len(one_epoch.index) > 4:
        x_wls, v_wls, cov_x, cov_v = wls.WLS_onePosition(one_epoch) # epoch like period 1s,10s or ....
        ecef_list.append((x_wls, v_wls, cov_x, cov_v))              # unit m, m/s and use tuple to store many variable 
ecef_list

[(array([-1626096.27650848,  5730704.25968617,  2271924.45408575]),
  array([-0.19996238, -0.12878232, -0.32954396]),
  array([[ 394.29931865, -106.03613622, -210.3347151 ],
         [-106.03613622, 1103.66820478,  364.63410236],
         [-210.3347151 ,  364.63410236,  442.74810839]]),
  array([[ 0.16134676, -0.21914449, -0.06975237],
         [-0.21914449,  0.87381625,  0.20926521],
         [-0.06975237,  0.20926521,  0.14174398]])),
 (array([-1626078.10276149,  5730643.85607048,  2271907.13850162]),
  array([-0.66876664,  2.56659827,  0.06400331]),
  array([[  612.45702385, -1218.90581455,  -524.55865003],
         [-1218.90581455,  3563.62877282,  1673.62070304],
         [ -524.55865003,  1673.62070304,   957.71055633]]),
  array([[ 0.10554468, -0.12034423, -0.05166195],
         [-0.12034423,  0.58452651,  0.17310991],
         [-0.05166195,  0.17310991,  0.1111434 ]])),
 (array([-1626091.4602989 ,  5730689.19483037,  2271948.49117996]),
  array([-0.12740886,  0.26918577, -0.556

In [11]:
# # Kalman Filter
# # Transistion ecef_list to each array numpy 
# x_wls = np.array([x[0] for x in ecef_list])
# v_wls = np.array([x[1] for x in ecef_list])
# cov_x = np.array([x[2] for x in ecef_list])
# cov_v = np.array([x[3] for x in ecef_list])
# x_kf, P_f = kalman_smoothing(x_wls, v_wls, cov_x, cov_v)
# ecef_list.append(x_kf)
# print(x_kf)  
# Kiểm tra và chuẩn hóa kích thước của các phần tử trong ecef_list
# max_len = max(len(x[0]) for x in ecef_list)

# # Hàm để chuẩn hóa kích thước của một phần tử
# def pad_to_max_len(array, max_len):
#     if len(array) < max_len:
#         pad_width = max_len - len(array)
#         array = np.pad(array, ((0, pad_width), (0, 0)), mode='constant', constant_values=0)
#     return array

# ecef_list_padded = [(pad_to_max_len(x[0], max_len), pad_to_max_len(x[1], max_len), pad_to_max_len(x[2], max_len), pad_to_max_len(x[3], max_len)) for x in ecef_list]

# Chuyển đổi ecef_list thành các mảng numpy riêng lẻ
# x_wls = np.array([x[0] for x in ecef_list_padded])
# v_wls = np.array([x[1] for x in ecef_list_padded])
# cov_x = np.array([x[2] for x in ecef_list_padded])
# cov_v = np.array([x[3] for x in ecef_list_padded])

x_wls = np.array([x[0] for x in ecef_list])
v_wls = np.array([x[1] for x in ecef_list])
cov_x = np.array([x[2] for x in ecef_list])
cov_v = np.array([x[3] for x in ecef_list])
# Áp dụng Kalman Filter
x_kf, P_f = kalman_smoothing(x_wls, v_wls, cov_x, cov_v)
ecef_list.append(x_kf)
print(x_kf, P_f)  # In ra giá trị ước lượng vị trí



[[-1626096.27650848  5730704.25968617  2271924.45408575]
 [-1626097.27385417  5730704.11687906  2271925.87047734]
 [-1626095.98226073  5730704.38716389  2271930.01170306]
 [-1626100.65799511  5730707.50553655  2271930.93109836]
 [-1626098.84907252  5730702.62384097  2271929.25826001]
 [-1626098.42599459  5730703.13150456  2271928.87781302]
 [-1626098.17390579  5730703.77802438  2271929.62341564]
 [-1626096.59826039  5730695.07375979  2271924.732801  ]
 [-1626089.90920311  5730675.57731306  2271927.9202052 ]
 [-1626092.1500156   5730671.67000273  2271925.5073195 ]
 [-1626096.65132761  5730664.82675983  2271918.35916116]
 [-1626095.17595982  5730665.12348944  2271918.51858431]
 [-1626095.39356936  5730664.55065958  2271917.95268441]
 [-1626097.14132741  5730663.71529368  2271919.04972648]
 [-1626097.1969278   5730663.86576043  2271918.92538415]
 [-1626087.50827194  5730641.53370472  2271909.17870629]
 [-1626088.55733041  5730644.81793502  2271909.56230478]
 [-1626086.00200913  5730645.96

In [12]:
# ecef_array = np.stack(ecef_list, axis=0)
# lla_array = np.stack(navpy.ecef2lla(ecef_array), axis=1)
# lla_array

# # Convert back to Pandas and save to csv
# lla_df = pd.DataFrame(lla_array, columns=['Latitude', 'Longitude', 'Altitude'])
# lla_df.to_csv('calculated_position_wls.csv')

# Giả sử ecef_list chứa các tuple với 4 phần tử: x_wls, v_wls, cov_x, cov_v
# Tách từng phần tử trong ecef_list thành các danh sách riêng biệt
x_wls_list = [x[0] for x in ecef_list]
# v_wls_list = [x[1] for x in ecef_list]
# cov_x_list = [x[2] for x in ecef_list]
# cov_v_list = [x[3] for x in ecef_list]

# Chuyển đổi các danh sách này thành các mảng numpy
x_wls_array = np.array(x_wls_list)
# v_wls_array = np.array(v_wls_list)
# cov_x_array = np.array(cov_x_list)
# cov_v_array = np.array(cov_v_list)

# Kiểm tra kích thước của các mảng
print(f"x_wls_array shape: {x_wls_array.shape}")
# print(f"v_wls_array shape: {v_wls_array.shape}")
# print(f"cov_x_array shape: {cov_x_array.shape}")
# print(f"cov_v_array shape: {cov_v_array.shape}")

# Nếu các mảng có kích thước phù hợp, ta có thể stack chúng lại
ecef_array = np.stack(x_wls_list, axis=0)
lla_array = np.stack(navpy.ecef2lla(ecef_array), axis=1)

# Chuyển đổi kết quả sang DataFrame và lưu thành file CSV
lla_df = pd.DataFrame(lla_array, columns=['Latitude', 'Longitude', 'Altitude'])
lla_df.to_csv('calculated_position_wls.csv', index=False)

print(lla_df.head())

x_wls_array shape: (204, 3)
    Latitude   Longitude    Altitude
0  21.004862  105.841373   74.942779
1  21.004920  105.841364    9.856561
2  21.005116  105.841368   68.801858
3  21.004971  105.841432  128.071494
4  21.004642  105.841396  -10.756490
